In [1]:
# Import the pandas library
import pandas as pd

In [2]:
# Read all excel files
Order_Report = pd.read_excel(r"C:\Users\nishc\OneDrive\Desktop\Study\Assignment\Company X - Order Report.xlsx")
Pincode_Zones = pd.read_excel(r"C:\Users\nishc\OneDrive\Desktop\Study\Assignment\Company X - Pincode Zones.xlsx")
SKU_Master = pd.read_excel(r"C:\Users\nishc\OneDrive\Desktop\Study\Assignment\Company X - SKU Master.xlsx")
Invoice = pd.read_excel(r"C:\Users\nishc\OneDrive\Desktop\Study\Assignment\Courier Company - Invoice.xlsx")
Rates = pd.read_excel(r"C:\Users\nishc\OneDrive\Desktop\Study\Assignment\Courier Company - Rates.xlsx")

In [3]:
# Remove duplicates from SKU Master and Pincode Zones
SKU_Master=SKU_Master.drop_duplicates()
Pincode_Zones=Pincode_Zones.drop_duplicates()
# Merge Order report and SKU master to calculate the total weight of orders
Order_Report = pd.merge(Order_Report, SKU_Master, on='SKU', how='left')
# Calculate the total weight of orders(Product wise)
Order_Report["Total_Weight"]=Order_Report["Weight (g)"]*Order_Report['Order Qty']
# Convert the weight from g to kg
Order_Report["Total_Weight"]=Order_Report["Total_Weight"]/1000

In [4]:
# Calculate the total weight of orders(Order wise)
Order_Weight = Order_Report.groupby(['ExternOrderNo'])['Total_Weight'].sum()

In [5]:
# Merge Order weight as per X and Zone as per X in Invoice provided by courier company 
Invoice=pd.merge(Invoice,Order_Weight, left_on='Order ID', right_on='ExternOrderNo')
Invoice = pd.merge(Invoice, Pincode_Zones, on=["Warehouse Pincode", "Customer Pincode"], suffixes=('_CC', '_X'))

In [6]:
# Convert zone in Rates to lower case for compatibility
Rates['Zone']=Rates['Zone'].str.lower()

In [7]:
# Merge Rates in invoice on zone as per X
Invoice = pd.merge(Invoice,Rates, left_on='Zone_X', right_on='Zone')

In [8]:
# Calculate total number of slab required for shipment of orders
Invoice['Total_Slabs']=Invoice['Total_Weight']/Invoice['Weight Slabs']
# Promoting total slabs column to next interger
Invoice['Total_Slabs']=Invoice['Total_Slabs'].apply(lambda x: int(x) if x == int(x) else int(x) + 1)

In [9]:
# Calculating additional Slabs required
Invoice['Additional_Slabs']=Invoice['Total_Slabs']-1

In [10]:
# Calculating Forward Additional Charges and RTO Additional Charges
Invoice['Forward Additional Charge']=Invoice['Forward Additional Weight Slab Charge']*Invoice['Additional_Slabs']
Invoice['RTO Additional Charge']=Invoice['RTO Additional Weight Slab Charge']*Invoice['Additional_Slabs']

In [11]:
# Calculating total charge as per type of Shipment
Invoice.loc[Invoice['Type of Shipment'] == "Forward charges", "Total Charge"] = Invoice['Forward Fixed Charge']+Invoice['Forward Additional Charge']
Invoice.loc[Invoice['Type of Shipment'] == "Forward and RTO charges", "Total Charge"] = Invoice['Forward Fixed Charge']+Invoice['Forward Additional Charge']+Invoice['RTO Fixed Charge']+Invoice['RTO Additional Charge']

In [12]:
# Calculating charge diffrence in total charge as per X and billing charge as per courier comapny
Invoice['Charge Diffrence']=Invoice['Billing Amount (Rs.)'].round(2)-Invoice['Total Charge'].round(2)
# calculating charge status to be over charged, under charged or correctly charged 
Invoice.loc[Invoice['Charge Diffrence']>0, 'Charge Status']= "Over Charged"
Invoice.loc[Invoice['Charge Diffrence']<0, 'Charge Status']= "Under Charged"
Invoice.loc[Invoice['Charge Diffrence']==0, 'Charge Status']= "Correctly Charged"

C:\Users\nishc\AppData\Local\Temp\ipykernel_14724\4610461.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Over Charged' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Invoice.loc[Invoice['Charge Diffrence']>0, 'Charge Status']= "Over Charged"


In [13]:
# Merge rates in invoice on zone as per courier company to get Weight Slabs as per courier company
Invoice = pd.merge(Invoice, Rates, left_on='Zone_CC', right_on='Zone', suffixes=(' as per X', ' as per CC'))

In [14]:
# Creating new dataframe to display required columns  
Output_Data_1=Invoice[['Order ID','AWB Code','Total_Weight','Weight Slabs as per X','Charged Weight', 'Weight Slabs as per CC','Zone_X','Zone_CC','Total Charge','Billing Amount (Rs.)','Charge Diffrence']].rename(columns={'Total_Weight':'Total Weight as per X', 'Charged Weight':'Total weight as per CC','Zone_X':'Zone as per X','Zone_CC':'Zone as per CC','Total Charge':'Expected Charge as per X','Billing Amount (Rs.)':'Charges Billed by CC'})

In [15]:
Output_Data_2=Invoice[['Charge Status', 'Charge Diffrence']]

In [16]:
Output_Data_2=Output_Data_2.groupby(['Charge Status'])['Charge Diffrence'].agg(['count','sum']).rename(columns={'count':'Count','sum':'Amount (Rs.)'})

In [17]:
Output_Data_1.to_excel('Output_Data_1.xlsx', index=False)

In [18]:
Output_Data_2.to_excel('Output_Data_2.xlsx')